In [1]:
import sys, os
import numpy as np
#%matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
import pydub
import torchaudio
import torch
import h5py as hf
import foundation as fd
import foundation.util as util

ModuleNotFoundError: No module named 'pydub'

In [ ]:
#root = '/home/fleeb/workspace/ml_datasets/audio/musicnet/'
root = '/mnt/d/Users/anwan/Documents/musicnet'

In [3]:
f = hf.File(os.path.join(root, 'musicnet.h5'), 'r')
list(f.keys())

['id_1727',
 'id_1728',
 'id_1729',
 'id_1730',
 'id_1733',
 'id_1734',
 'id_1735',
 'id_1739',
 'id_1742',
 'id_1749',
 'id_1750',
 'id_1751',
 'id_1752',
 'id_1755',
 'id_1756',
 'id_1757',
 'id_1758',
 'id_1759',
 'id_1760',
 'id_1763',
 'id_1764',
 'id_1765',
 'id_1766',
 'id_1768',
 'id_1771',
 'id_1772',
 'id_1773',
 'id_1775',
 'id_1776',
 'id_1777',
 'id_1788',
 'id_1789',
 'id_1790',
 'id_1791',
 'id_1792',
 'id_1793',
 'id_1805',
 'id_1807',
 'id_1811',
 'id_1812',
 'id_1813',
 'id_1817',
 'id_1818',
 'id_1819',
 'id_1822',
 'id_1824',
 'id_1828',
 'id_1829',
 'id_1835',
 'id_1859',
 'id_1872',
 'id_1873',
 'id_1876',
 'id_1893',
 'id_1916',
 'id_1918',
 'id_1919',
 'id_1922',
 'id_1923',
 'id_1931',
 'id_1932',
 'id_1933',
 'id_2075',
 'id_2076',
 'id_2077',
 'id_2078',
 'id_2079',
 'id_2080',
 'id_2081',
 'id_2082',
 'id_2083',
 'id_2104',
 'id_2105',
 'id_2106',
 'id_2112',
 'id_2113',
 'id_2114',
 'id_2116',
 'id_2117',
 'id_2118',
 'id_2119',
 'id_2127',
 'id_2131',
 'id

In [4]:
f['id_1788']['data'].shape

(22631758,)

In [5]:
sample = f['id_1788']['data'].value
sample.shape

(22631758,)

In [371]:
f.close()

In [6]:
sec = sample.size / 44100
sec

513.19179138322

In [7]:
x = torch.tensor(sample).float()
x.size(), x.max(), x.min()

(torch.Size([22631758]), tensor(0.7954), tensor(-0.8617))

In [151]:
fs = 44100
ms = fs // 1000

In [137]:
torchaudio.transforms.MEL2?

In [322]:
ws = 40 * ms
hop = 10 * ms
n_mels = 100
spec = torchaudio.transforms.MEL2(fs, ws=ws, hop=hop, n_mels=n_mels)
print(fs, ws, hop, n_mels)

44100 1760 440 100


In [323]:
y = spec(x.unsqueeze(0))
y.size()

torch.Size([1, 51432, 100])

In [324]:
y.size(1) / sec # width of a 1 sec segment

100.21984151650967

In [325]:
plt.figure()

In [326]:
size = 500
step = 1
i=0

In [327]:
#plt.switch_backend('TkAgg')

In [353]:
plt.cla()
plt.axis('off')
b = i*size
i+=1
e = i*size
plt.title('{}:{}'.format(b,e))
plt.imshow(y[0,b:e:step].numpy().T)
plt.tight_layout()

In [3]:
# meta data
with open(os.path.join(root, 'musicnet_metadata.csv'), 'r') as f:
    lines = f.readlines()
len(lines)

331

In [4]:
meta = {}
times = {}
for line in lines[1:]:
    tokens = line.split(',')
    ID = int(tokens[0])
    meta[ID] = tokens[1][1:-1]
    times[ID] = int(tokens[-1])
len(meta), len(times)

(330, 330)

In [5]:
set(meta.values())

{'Bach',
 'Beethoven',
 'Brahms',
 'Cambini',
 'Dvorak',
 'Faure',
 'Haydn',
 'Mozart',
 'Ravel',
 'Schubert'}

In [6]:
cats = ['baroque', 'classical', 'romantic', 'modern']

labels = {n:i for i, n in enumerate(cats)}

styles = {
    'Bach':'baroque',
    'Beethoven':'romantic',
    'Brahms':'romantic',
    'Cambini': 'classical',
    'Dvorak': 'romantic',
    'Faure': 'modern',
    'Ravel': 'modern',
    'Haydn': 'classical',
    'Mozart': 'classical',
    'Schubert': 'romantic',
}

In [7]:
idlbls = {}
for ID, name in meta.items():
    idlbls[ID] = labels[styles[name]]
len(idlbls)

330

In [8]:
for ID, lbl in idlbls.items():
    print(ID, lbl)
    break

1727 2


In [9]:
s = []
w = []
for ID in idlbls.keys():
    s.append(idlbls[ID])
    w.append(times[ID])

In [10]:
np.bincount(s, weights=w)

array([11041., 12851., 95211.,  3606.])

In [11]:
fail
with hf.File(os.path.join(root, 'musicnet.h5'), 'r+') as f:
    for name in f.keys():
        ID = int(name[3:])
        f[name].attrs['label'] = idlbls[ID]
        f[name].attrs['style'] = cats[idlbls[ID]]
        f[name].attrs['composer'] = meta[ID]

In [12]:
with hf.File(os.path.join(root, 'musicnet.h5'), 'r+') as f:
    for name in f.keys():
        print(f[name].attrs['style'], f[name].attrs['composer'])

romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
romantic Schubert
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
classical Mozart
romantic Dvorak
romantic Dvorak
romantic Dvorak
roma